## Proto type for the ASAG as a Jupyter file


### Importing packeges

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import torch
from nltk.tokenize import word_tokenize
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import scipy
import numpy as np
import scipy.spatial
from nltk.tokenize import word_tokenize




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Pre Processing

In [4]:
class PreProcess:
    def __init__(self):
        pass

    def tokenization(self, question, answer):
        question_tokens = word_tokenize(question)
        answer_tokens = word_tokenize(answer)
        return question_tokens, answer_tokens

    def question_demoting(self, question, answer):

        question_tokens, answer_tokens = self.tokenization(question, answer)
        demoted_tokens = [word for word in answer_tokens if word not in question_tokens]
        return demoted_tokens

    def remove_stop_words(self, demoted_tokens):

        stop_words = set(stopwords.words('english'))
        filtered_sentence = [w for w in demoted_tokens if not w in stop_words]
        return filtered_sentence


def pre_processing(ques, ans):
    """
        Preprocess question and answer. Returns the filtered list of tokens
    :param ques: string
    :param ans: string
    :return: list
        Returns the filtered list after all preprocessing steps
    """
    preprocess = PreProcess()
    question_demoted = preprocess.question_demoting(ques, ans)
    filtered_sentence = preprocess.remove_stop_words(question_demoted)
    return filtered_sentence



### Scentence vector 

In [5]:
class Embedding2Array:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('./bert_tokenizer')
        self.model = BertModel.from_pretrained('./bert_model')
        # self.model_name = "tum-nlp/NegBLEURT"
        # self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        # self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
        

    def bert(self, sentence):
        inputs = self.tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # print(outputs.last_hidden_state)
        embeddings = outputs.last_hidden_state
        word_array = []
        for i in range(embeddings.shape[1]):
            word_array.append(embeddings[0, i].numpy())

        return word_array


In [6]:
class SentenceEmbeddings:
    def __init__(self):
        pass

    def sowe(self, array):

        return sum(array)

    def mowe(self, array):

        return sum(array)/len(array)

    def gpt_sowe(self, array):

        return [sum(i) for i in zip(*array)]


In [7]:
class SimilarityFunctions:

    def __init__(self, word_array_1, word_array_2):

        self.word_array_1 = word_array_1
        self.word_array_2 = word_array_2

    def get_cosine_similarity(self,u,v):
        return 1-scipy.spatial.distance.cosine(u,v)

    def __cosine_similarity_matrix(self):
        '''
        Creates a matrix depicting the cosine distances between the words of two sentences
        returns: array
          Similarity matrix of words in two sentences
        '''
        matrix = np.zeros((len(self.word_array_1), len(self.word_array_2)))

        for i in range(0, len(self.word_array_1)):
            for j in range(0, len(self.word_array_2)):
                matrix[i][j] = self.get_cosine_similarity(self.word_array_1[i], self.word_array_2[j])
        return matrix.T


    def get_similar_words(self, sentence_1, sentence_2):
        '''Prints similar word from second sentence for each word in the first sentence
        returns: list of similar words
        '''

        token_1 = word_tokenize(sentence_1)
        token_2 = word_tokenize(sentence_2)

        similarity_matrix = self.__cosine_similarity_matrix()

        similar_word_dict = {}
        for row in range(0, len(similarity_matrix[0])):

            min_val = min(similarity_matrix.T[row])  # Here min value of transpose is found. To understand it print similarity matrix and find the logic
            index = (np.where(similarity_matrix.T[row] == min_val))[0]
            similar_word_list = []

            for i in range(0, len(index)):
                similar_word_list.append(token_2[index[i]])
            similar_word_dict[token_1[row]] = similar_word_list

        return ( similar_word_dict)

In [55]:
def evaluate(question,key,answer):
    if len(answer) == 0:
        return 0
    # Preprocess student answer
    pp_desired = pre_processing(question, key)
    pp_student = pre_processing(question, answer)
    if len(pp_desired) == 0 :
        pp_student = pre_processing('', answer)
        pp_desired = pre_processing('', key)
    elif len(pp_student) == 0:
        return 0
    
    # Assign embeddings to desired answer and student answer
    embed2arr = Embedding2Array()

    word_array_1 = embed2arr.bert(pp_desired)
    word_array_2 = embed2arr.bert(pp_student)
    # print(word_array_1)

    # Compare and assign cosine similarity to the answers

    similarity_tools = SimilarityFunctions(word_array_1, word_array_2)
    sentence_embed = SentenceEmbeddings()

    text_1_embed = sentence_embed.sowe(word_array_1)
    text_2_embed = sentence_embed.sowe(word_array_2)
    # print(text_1_embed)

    bert_similarity_score = similarity_tools.get_cosine_similarity(text_1_embed, text_2_embed)
    return bert_similarity_score

In [9]:
evaluate('What is the role of a prototype program in problem solving? <STOP>',
         'To simulate the behaviour of portions of the desired software product.<STOP>',
         'High risk problems are address in the prototype program to make sure that the program is feasible. <STOP> A prototype may also be used to show a company that the software can be possibly programmed. <STOP>')

0.6935483325054561


In [10]:
import pandas as pd
from Embeddings import preprocess
df = preprocess.pair(pd.read_csv("data/qa.csv"))
df.head()

All modules imported successfully


,Question,Response,Key,Averages,Medians,Other_scores
0,What is the role of a prototype program in pro...,High risk problems are address in the prototyp...,To simulate the behaviour of portions of the d...,3.5,4.0,3.0
1,What is the role of a prototype program in pro...,To simulate portions of the desired final prod...,To simulate the behaviour of portions of the d...,5.0,5.0,5.0
2,What is the role of a prototype program in pro...,A prototype program simulates the behaviors of...,To simulate the behaviour of portions of the d...,4.0,5.0,3.0
3,What is the role of a prototype program in pro...,Defined in the Specification phase a prototype...,To simulate the behaviour of portions of the d...,5.0,5.0,5.0
4,What is the role of a prototype program in pro...,It is used to let the users have a first idea ...,To simulate the behaviour of portions of the d...,3.0,3.0,3.0


In [53]:
print(x.loc[2240, 'Question'])
print(x.loc[2240, 'Response'])
print(x.loc[2240, 'Key'])
pre_processing('', x.loc[2240, 'Response'])

Order the following functions by their running time: n to the power of 2; log -LRB- log n -RRB- ; 2 to the power of -LRB- log n -RRB- ; n!; n to the power of 3. <STOP>
2 to the power of -LRB- log n -RRB- n to the power of 2 to the power of 3 log -LRB- long n -RRB- n! <STOP>
log -LRB- log n -RRB- ; 2 to the power of -LRB- log n -RRB- ; n to the power of 2; n to the power of 3; n! <STOP>


['2',
 'power',
 '-LRB-',
 'log',
 'n',
 '-RRB-',
 'n',
 'power',
 '2',
 'power',
 '3',
 'log',
 '-LRB-',
 'long',
 'n',
 '-RRB-',
 'n',
 '!',
 '<',
 'STOP',
 '>']

In [57]:
for index, row in x.iterrows():
    print(index, row["Averages"], round(evaluate(row["Question"], row["Key"], row["Response"]), 1) * 5)
    

31 4.5 3.0
1925 2.5 3.5
1519 5.0 4.0
2240 3.25 3.5
1523 5.0 3.5
1550 5.0 5.0
1479 3.0 4.0
661 3.0 3.5
1032 4.0 3.5
898 3.0 3.5
227 2.0 4.0
1878 5.0 4.0
797 5.0 3.0
1249 5.0 4.5
1979 5.0 4.0
1109 5.0 4.5
118 5.0 5.0
943 2.0 3.0
512 4.5 4.0
441 5.0 4.5
1666 5.0 5.0
1603 3.0 4.0
595 5.0 4.0
1928 5.0 3.5
905 3.0 3.5
13 4.5 5.0
967 5.0 4.0
50 5.0 4.0
243 4.5 3.5
503 5.0 4.0
1673 5.0 5.0
1283 3.5 4.0
2232 4.0 3.5
1279 3.5 4.0
2360 3.0 4.0
222 5.0 3.5
2438 1.5 2.5
1995 5.0 5.0
336 5.0 4.0
1756 5.0 5.0
98 5.0 4.5
1009 4.0 3.5
1471 5.0 3.5
2220 5.0 5.0
1113 3.5 3.0
2078 4.0 4.5
1239 5.0 4.5
659 4.0 4.0
1241 5.0 4.5
370 4.5 5.0


In [58]:
x

,Question,Response,Key,Averages,Medians,Other_scores
31,What stages in the software life cycle are inf...,"Verification, coding, refining the solution an...",The testing stage can influence both the codin...,4.50,4.0,5.0
1925,How are objects initialized when they are crea...,Objects are initialized by a value given. <STO...,By using constructors. <STOP>,2.50,4.0,5.0
1519,What is a queue? <STOP>,"A first in, first out Abstract Data type that ...","A data structure that can store elements, whic...",5.00,5.0,5.0
2240,Order the following functions by their running...,2 to the power of -LRB- log n -RRB- n to the p...,log -LRB- log n -RRB- ; 2 to the power of -LRB...,3.25,4.0,9.0
1523,What is a queue? <STOP>,a finite ordered list with zero or more elemen...,"A data structure that can store elements, whic...",5.00,5.0,5.0
1550,What are the two main functions defined by a q...,"enqueue, dequeue <STOP>",enqueue and dequeue <STOP>,5.00,5.0,5.0
1479,What operations would you need to perform to f...,You would have to keep using peek or getTop to...,Pop all the elements and store them on another...,3.00,2.0,4.0
661,What is the main difference between strings de...,the type string uses less storage and you have...,The strings declared using an array of charact...,3.00,1.0,5.0
1032,How can an array be addressed in pointer or of...,Element b [n] can be accessed by multiplied by...,By initializing a pointer to point to the firs...,4.00,5.0,3.0
898,What is the number of operations for insertion...,"The number of operations for insertions sort, ...",N -LRB- the length of the array -RRB- operatio...,3.00,3.0,3.0
